In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
from collections import Counter


%matplotlib inline

In [18]:
data = pd.read_csv("../hispanic_colrect/final_with_labels.csv")
data_matrix = data.as_matrix()

In [19]:
data.head()

,ID,pubcsnum,mar_stat,sex,primsite,histo3v,beho3v,grade,dx_conf,csexten,...,maligcount,benbordcount,year_dx,nhiade,codpub,srv_time_mon,srv_time_mon_flag,surv_1,surv_2,surv_5
0,1,7011365,9,2,C184,8210,3,2,1,200,...,4,0,2010,7,0,71,1,1,1,1
1,2,7014780,5,2,C209,8140,3,2,1,200,...,2,0,2007,6,21040,36,1,1,1,0
2,3,7017183,2,1,C186,8140,2,9,1,50,...,3,0,2012,6,21050,37,1,1,1,0
3,4,7017183,2,1,C209,8263,3,2,1,455,...,3,0,2012,6,21050,37,1,1,1,0
4,5,7018423,2,1,C186,8263,3,3,1,400,...,3,0,2011,2,0,52,1,1,1,1


In [10]:
data_matrix.shape

(37575, 30)

In [20]:
data_matrix[0]

array([1, 7011365, 9, 2, 'C184', 8210, 3, 2, 1, 200, 0, 0, 3.0, 3.0, 0.0,
       50, 0, 1, 1502, 93, 0, 5, 30, 4, 0, 2010, 7, 0, 71, 1, 1, 1, 1], dtype=object)

In [16]:
data_matrix[:,22]

array([4, 2, 3, ..., 1, 1, 1], dtype=object)

# Preprocessing

In [13]:
Y1 = data_matrix[:,-3]
Y2 = data_matrix[:,-2]
Y5 = data_matrix[:,-1]

In [39]:
X = data_matrix[:,2:24]
X_cat = X[:,:-5]
# primary site is only categorical variable with string labels
X_primsite = X[:,2]
X_cont = X[:,-5:]
# X_cat contains only categorical variables with integer labels
X_cat = np.delete(X_cat, [2], axis=1)

In [40]:
Fill missing values in three eval columns with 9: unknown
X_cat[:,9] = np.array([9 if np.isnan(x) else x for x in X_cat[:,9]])
X_cat[:,10] = np.array([9 if np.isnan(x) else x for x in X_cat[:,10]])
X_cat[:,11] = np.array([9 if np.isnan(x) else x for x in X_cat[:,11]])

In [41]:
# one-hot encode primary site
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
X_primsite_1hot = encoder.fit_transform(X_primsite)
encoder.classes_

array(['C180', 'C181', 'C182', 'C183', 'C184', 'C185', 'C186', 'C187',
       'C188', 'C189', 'C199', 'C209', 'C260'], dtype='<U4')

In [42]:
# one-hot encode remaining categorical variables
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
X_cat_1hot = encoder.fit_transform(X_cat)

In [43]:
# combine all categorical variables
X_cat = np.hstack((X_primsite_1hot, X_cat_1hot))

The continuous variables in order: age_dx, eod10_pn, eod10_ne, cstumsiz, maligcount. Age and maligcount do not require imputation, and eod10_ne (nodes examined) will not be used as a variable.


Here we will impute values for eod10_pn and cstumsiz. We will implement multivariate imputation by chained equations (MICE) on these two continuous variables. Their indices are and values which need to be imputed are:

1: eod10_pn (95, 97, 98, 99)<br>
3: cstumsiz (998, 999)<br>

Additionally, we will replace the values (991, 992, 993, 994, 995) in cstumsiz with (1, 2, 3, 4, 5) respectively.


### Note: The above should be re-verified for each different dataset. This is for colorectal cancer in Hispanic patients after 2004.

In [44]:
# implements MICE imputation with linear regression model on eod10_pn and cstumsiz
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LinearRegression

NUM_CYCLES=10

missing_values_pn = (95, 97, 98, 99)
missing_values_tumsiz = (998, 999)

pn = np.array(X_cont[:,1]).reshape(-1,1)
tumsiz = np.array(X_cont[:,3]).reshape(-1,1)

# non-MICE imputation
for i, size in enumerate(tumsiz):
    if size not in (990, 991, 992, 993, 994, 995):
        continue
    elif size == 990:
        tumsiz[i] = 0
    elif size == 991:
        tumsiz[i] = 1
    elif size == 992:
        tumsiz[i] = 2
    elif size == 993:
        tumsiz[i] = 3
    elif size == 994:
        tumsiz[i] = 4
    elif size == 995:
        tumsiz[i] = 5


# set missing values to -1
for i, value in enumerate(pn):
    if value in missing_values_pn:
        pn[i] = -1
for i, value in enumerate(tumsiz):
    if value in missing_values_tumsiz:
        tumsiz[i] = -1

# indices for valid and imputation-needed positions in pn and tumsiz
idx_pn_missing = np.where(pn == -1)[0]
idx_pn_valid = np.where(pn != -1)[0]
idx_tumsiz_missing = np.where(tumsiz == -1)[0]
idx_tumsiz_valid = np.where(tumsiz != -1)[0]
    
# set missing values to mean
# copy = false modifies array in place
imp_pn = Imputer(missing_values = -1, copy=False)
imp_pn.fit_transform(pn)
imp_tumsiz = Imputer(missing_values = -1, copy=False)
imp_tumsiz.fit_transform(tumsiz)


for i in np.arange(NUM_CYCLES):
    print("Cycle " + str(i+1))
    
    # fit pn
    X_ind = np.hstack((X_cat[idx_pn_valid], tumsiz[idx_pn_valid]))
    X_rep = np.hstack((X_cat[idx_pn_missing], tumsiz[idx_pn_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, pn[idx_pn_valid])
    pn[idx_pn_missing] = lin_reg.predict(X_rep)
    pn = np.maximum(pn, 0)
    pn = np.minimum(pn, 100)
    
    # fit tumsiz
    X_ind = np.hstack((X_cat[idx_tumsiz_valid], pn[idx_tumsiz_valid]))
    X_rep = np.hstack((X_cat[idx_tumsiz_missing], pn[idx_tumsiz_missing]))
    lin_reg = LinearRegression()
    lin_reg.fit(X_ind, tumsiz[idx_tumsiz_valid])
    tumsiz[idx_tumsiz_missing] = lin_reg.predict(X_rep)
    tumsiz = np.maximum(tumsiz, 0)
    tumsiz = np.minimum(tumsiz, 990)


Cycle 1
Cycle 2
Cycle 3
Cycle 4
Cycle 5
Cycle 6
Cycle 7
Cycle 8
Cycle 9
Cycle 10


In [45]:
print(np.mean(pn[idx_pn_valid]))
print(np.std(pn[idx_pn_valid]))
print(np.mean(pn[idx_pn_missing]))
print(np.std(pn[idx_pn_missing]))

1.9226782686586263
3.956180991058403
1.6075881452854033
9.259678595855577


In [46]:
print(np.mean(tumsiz[idx_tumsiz_valid]))
print(np.std(tumsiz[idx_tumsiz_valid]))
print(np.mean(tumsiz[idx_tumsiz_missing]))
print(np.std(tumsiz[idx_tumsiz_missing]))

46.73218773744813
40.93510963587236
44.220240837237405
64.46177508671394


In [47]:
age_dx = np.array(X_cont[:,0])
malig = np.array(X_cont[:,4])
X = np.column_stack((X_cat, age_dx, pn, malig, tumsiz))
print(X.shape)

(46785, 313)


We save the preprocessed data to disk. They are now available for our models.

In [48]:
np.save('../hispanic_colrect/X.npy', X)
np.save('../hispanic_colrect/Y5.npy', Y5)
np.save('../hispanic_colrect/Y2.npy', Y2)
np.save('../hispanic_colrect/Y1.npy', Y1)